<a href="https://colab.research.google.com/github/ZainAli24/Quater_3_online_classes/blob/main/Function_calling_online_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Function calling config

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Function_calling_config.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

Specifying a `function_calling_config` allows you to control how the Gemini API acts when `tools` have been specified. For example, you can choose to only allow free-text output (disabling function calling), force it to choose from a subset of the functions provided in `tools`, or let it act automatically.

This guide assumes you are already familiar with function calling. For an introduction, check out the [docs](https://ai.google.dev/docs/function_calling).

In [2]:
!pip install -U -q "google-generativeai>=0.7.2"

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [3]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get("Gemini_API"))

## Set up a model with tools

This example uses 3 functions that control a simple hypothetical lighting system. Using these functions requires them to be called in a specific order. For example, you must turn the light system on before you can change color.

While you can pass these directly to the model and let it try to call them correctly, specifying the `function_calling_config` gives you precise control over the functions that are available to the model.

In [4]:
def enable_lights():
    """Turn on the lighting system."""
    print("LIGHTBOT: Lights enabled.")


def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")


def stop_lights():
    """Stop flashing lights."""
    print("LIGHTBOT: Lights turned off.")


light_controls = [enable_lights, set_light_color, stop_lights]
instruction = "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color. Do not perform any other tasks."

model = genai.GenerativeModel(
    "models/gemini-1.5-pro", tools=light_controls, system_instruction=instruction
)

chat = model.start_chat()

Create a helper function for setting `function_calling_config` on `tool_config`.

In [5]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

 `content_types.to_tool_config` ka **matlab yahi hai**:  

1. **AI Model ko batana hai ke tool ko kaise call karna hai** →  
   - `"auto"` → AI **khud** se function call kare.  
   - `"manual"` → AI **sirf tab call kare jab user kahe**.  

2. **AI ko batana hai ke sirf ye specific tools allowed hain** →  
   - Example: `["fetch_weather", "calculate_sum"]`  


💡 **Simple words mein:**  
**content_types.to_tool_config :** Ye ek **configuration hai jo AI model ko instructions deti hai** ke kaunse tools use karne hain aur kis mode mein call karne hain! 🚀

## Text-only mode: `NONE`

"`none`" ka matlab → AI sirf text-based jawab dega, tools use nahi karega.
Agar tools diye bhi ho tab bhi AI unko ignore karega.
Ye sirf ek normal chatbot ki tarah behave karega, bina kisi extra function ke.

In [7]:
tool_config = tool_config_from_mode("none")

response = chat.send_message(
    "Hello light-bot, what can you do?", tool_config=tool_config
)
print(response.text)

TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).

## Automatic mode: `AUTO`

"`AUTO`" ka matlab: AI khud decide karega ke function call kare ya sirf text bheje.
Agar zaroorat ho to tool use karega, warna simple text response dega.
AI ko puri azaadi hoti hai ke response kaise dena hai.

In [8]:
tool_config = tool_config_from_mode("auto")

response = chat.send_message("Light this place up!", tool_config=tool_config)
print(response.parts[0])

TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).

## Function-calling mode: `ANY`

"`ANY`" mode ka matlab hai AI ko force karna ke wo function call kare.
Agar tum allowed functions specify karte ho, to AI sirf unhi functions me se ek choose karega.
Agar tum allowed functions specify nahi karte, to AI kisi bhi available function ko choose kar sakta hai.
`Example`: Agar lights already on hain, to AI lights ko off karne ka function choose karega.

In [ ]:
available_fns = ["set_light_color", "stop_lights"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("Make this place PURPLE!", tool_config=tool_config)
print(response.parts)

## Automatic function calling
`enable_automatic_function_calling = True` ka matlab hai AI ko automatic function calling ke liye enable kar diya gaya hai, yani AI apne aap function call karega jab zaroorat ho.

Yahan pe AI ke paas ek hi function available hai, "`enable_lights`".

Message "It's awful dark in here..." hone par AI apne aap lights ko on kar dega.

In [17]:
available_fns = ["enable_lights"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
response = auto_chat.send_message("It's awful dark in here...", tool_config=tool_config)
print(response.text)

BadRequest: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: `allowed_function_names` should be a subset of the provided `function_declarations` names. Found invalid function names: enable_lights

In [ ]:
available_fns = ["enable_lights" , "set_light_color", "stop_lights"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
response = auto_chat.send_message("It's awful dark in here - make it orange and then switch off the lights", tool_config=tool_config)
print(response.text)

In [ ]:
available_fns = ["enable_lights" , "set_light_color", "stop_lights"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
response = auto_chat.send_message("Make this place blue", tool_config=tool_config)
print(response.text)

## Further reading

Check out the function calling [quickstart](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb) for an introduction to function calling. You can find another fun function calling example [here](https://github.com/google-gemini/cookbook/blob/main/quickstarts/rest/Function_calling_REST.ipynb) using curl.




### **Pehla Code:**  
```python
tool_config = tool_config_from_mode("auto")

response = chat.send_message("Light this place up!", tool_config=tool_config)
print(response.parts[0])
```
**Output:**  
```
function_call {
  name: "enable_lights"
  args {
  }
}
```


### **Dusra Code:**  
```python
available_fns = ["enable_lights"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
auto_chat.send_message("It's awful dark in here...", tool_config=tool_config)
```
**Output:**  
```
Light Enable
```

- **Pehle wale code mein:**  
  - Model **sirf decide kar raha hai** ke kaunsa function call karna hai (`enable_lights`).  
  - **Lekin function ka response nahi le raha**, sirf function call ka structure dikh raha hai.  

- **Dosre wale code mein:**  
  - Model **function ko call bhi kar raha hai** aur **uska response bhi return kar raha hai** (`Light Enable`).  
  - Is wajah se function ka **result** bhi output mein dikh raha hai.  

✅ **Conclusion:**  
- **AUTO mode:** Model **sirf function ko call karne ka faisla karta hai**.  
- **ANY mode + auto function calling:** Model **function ko call bhi karta hai aur uska result bhi print karta hai**.  

## **Second Example :** Room light and Room temprature

In [9]:
from typing import Literal
def set_light_values(brightness: int, color_temp: Literal["daylight", "cool", "warm"]):
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {
        "brightness": brightness,
        "colorTemperature": color_temp
    }

In [10]:
model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[set_light_values])

In [11]:
chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message('Dim the lights so the room feels cozy and warm.')
response.text

"OK. I've dimmed the lights to 30% brightness and set the color temperature to warm.  Is there anything else?\n"

## **Third Example :** Parallel function calling

In [12]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."

# {energetic=} ka matlab hai variable ka naam bhi print ho aur uski value bhi.

def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

# :.0% → Yeh formatting rule hai:
# : → Formatting start karne ke liye.
# .0 → Decimal places ka count (yahan 0 ka matlab point ke baad koe value nahe ae gi).
# % → Iska matlab hai value ko 100 se multiply karke percentage format mein dikhana.

In [14]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party!")
# response.text
# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(loud=True, energetic=True, bpm=120.0)
dim_lights(brightness=0.5)


In [16]:
# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Never gonna give you up.",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    genai.protos.Part(function_response=genai.protos.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

Ok, I've turned on the disco ball, started playing some energetic music at 120 bpm, and dimmed the lights to 50% brightness. Let's get this party started!



# **Function Information:** ke model ke pass kis tarah se function ja rahe hai

In [15]:
model._tools.to_proto()

[function_declarations {
   name: "power_disco_ball"
   description: "Powers the spinning disco ball."
   parameters {
     type_: OBJECT
     properties {
       key: "power"
       value {
         type_: BOOLEAN
       }
     }
     required: "power"
   }
 }
 function_declarations {
   name: "start_music"
   description: "Play some music matching the specified parameters.\n\n    Args:\n      energetic: Whether the music is energetic or not.\n      loud: Whether the music is loud or not.\n      bpm: The beats per minute of the music.\n\n    Returns: The name of the song being played.\n    "
   parameters {
     type_: OBJECT
     properties {
       key: "loud"
       value {
         type_: BOOLEAN
       }
     }
     properties {
       key: "energetic"
       value {
         type_: BOOLEAN
       }
     }
     properties {
       key: "bpm"
       value {
         type_: INTEGER
       }
     }
     required: "energetic"
     required: "loud"
     required: "bpm"
   }
 }
 functi